In [23]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import pickle
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import normalize
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm

In [19]:
df = pd.read_csv("../data/train.csv").drop("id",axis=1)
df_test = pd.read_csv("../data/test.csv").drop("id",axis=1)

labels = df["target"].values
print(type(labels))
print(labels.shape)

unique_labels = np.unique(labels)
nb_labels = len(unique_labels)

df_train = df.drop("target", axis=1)

X = df_train.values
X_test = df_test.values

print(X.shape)
print(X_test.shape)

y = []
for label in labels:
    y.append(int(label[-1:])-1)
y = np.asarray(y)


print("Construction des centroïdes")
centroids = []
for label in unique_labels:
    same_labels = df_train.loc[df['target'] == label]
    same_labels = same_labels.values
    centroids.append(same_labels.sum(axis = 0)/same_labels.shape[0])
centroids = np.asarray(centroids)


print("Construction de la représentation dcDistance")

X_dc = pairwise_distances(X,centroids)
X_test_dc = pairwise_distances(X_test, centroids, metric='minkowski')

print("Construction terminée \n")

y_train = y

<class 'numpy.ndarray'>
(61878,)
(61878, 93)
(144368, 93)
Construction des centroïdes
Construction de la représentation dcDistance
Construction terminée 



In [20]:
X_norm = Normalizer().fit_transform(X_dc)

In [21]:
X_norm = Normalizer().fit_transform(X_dc)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2,
random_state=42)

target_names = ['class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

In [22]:
clf_tmp = ExtraTreesClassifier(n_estimators=250, random_state=0, n_jobs=-1)
clf = CalibratedClassifierCV(clf_tmp, method='isotonic', cv=5)
clf.fit(X_train, y_train)
print("Fit terminé")
pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

Fit terminé
0.65
             precision    recall  f1-score   support

    class_1       0.57      0.23      0.33       399
    class_2       0.66      0.85      0.74      3178
    class_3       0.51      0.36      0.42      1561
    class_4       0.62      0.14      0.22       538
    class_5       0.96      0.95      0.96       565
    class_6       0.90      0.91      0.91      2884
    class_7       0.62      0.62      0.62       552
    class_8       0.82      0.87      0.84      1674
    class_9       0.79      0.82      0.80      1025

avg / total       0.74      0.75      0.73     12376



In [5]:
# Knn nature 
#clf = KNeighborsClassifier(n_neighbors=200, n_jobs=3)
clf = SVC(kernel='linear', C=0.4)
%time clf.fit(X_train, y_train)
#%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
#print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

CPU times: user 51.1 s, sys: 836 ms, total: 51.9 s
Wall time: 52.1 s
Accuracy : 0.66
             precision    recall  f1-score   support

    class_1       0.00      0.00      0.00       399
    class_2       0.53      0.96      0.68      3178
    class_3       0.00      0.00      0.00      1561
    class_4       0.00      0.00      0.00       538
    class_5       0.93      0.92      0.92       565
    class_6       0.88      0.85      0.87      2884
    class_7       0.87      0.20      0.32       552
    class_8       0.62      0.79      0.69      1674
    class_9       0.69      0.69      0.69      1025

avg / total       0.56      0.66      0.58     12376



/Users/nour/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# Knn calibré


In [ ]:
clf = MultinomialNB()
X_train_nb = Normalizer().fit_transform(X_train)
X_test_nb = Normalizer().fit_transform(X_test)
%time clf.fit(X_train_nb, y_train)
%time pred = clf.predict_proba(X_test_nb)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

In [16]:
#clf_tmp_1 = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
#clf_tmp_2 = CalibratedClassifierCV(clf_tmp_1, method='isotonic', cv=5)
#clf = AdaBoostClassifier(clf_tmp_1,
#    n_estimators=10,
#    learning_rate=1.0)

clf_tmp = KNeighborsClassifier(n_neighbors=300, n_jobs=-1)
clf = CalibratedClassifierCV(clf_tmp, method='isotonic', cv=5)
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

CPU times: user 19.8 s, sys: 313 ms, total: 20.1 s
Wall time: 7.17 s
CPU times: user 24.3 s, sys: 200 ms, total: 24.5 s
Wall time: 8.48 s
Kaggle log loss metric : 0.75
Accuracy : 0.71
             precision    recall  f1-score   support

    class_1       0.31      0.08      0.12       399
    class_2       0.62      0.86      0.72      3178
    class_3       0.47      0.28      0.35      1561
    class_4       0.62      0.03      0.06       538
    class_5       0.90      0.95      0.92       565
    class_6       0.88      0.87      0.87      2884
    class_7       0.60      0.51      0.55       552
    class_8       0.75      0.83      0.79      1674
    class_9       0.71      0.78      0.74      1025

avg / total       0.69      0.71      0.67     12376



In [17]:
pickle.dump((clf, pred), open('../data/knn300.p', 'wb'))

In [ ]:
print(X_train.shape)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))


In [ ]:
f = RandomForestClassifier(n_estimators=100, n_jobs=-1)
clf = CalibratedClassifierCV(f, method='isotonic', cv=5)
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

In [ ]:
clf_tmp = ExtraTreesClassifier(n_estimators=250, random_state=0, n_jobs=-1)
clf = CalibratedClassifierCV(clf_tmp, method='isotonic', cv=5)
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

In [ ]:
# A supprimer
clf = ExtraTreesClassifier(n_estimators=150, min_samples_split=2, random_state=0, n_jobs=-1)
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

# Methodes longues (feature reduction)

In [ ]:
print(X_dc.shape)

In [ ]:
# Feature selection
#k_bests = np.arange(5, 90, step=5)
k_bests=[2]
#clf = MultinomialNB()
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

for k_best in k_bests:
    selection = SelectKBest(chi2, k=k_best)
    X_new = selection.fit_transform(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2,
                                                        random_state=42)
    
    clf.fit(X_train, y_train)
    pred = clf.predict_proba(X_test)
    print('--- k_best = '+str(k_best))
    print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))


In [ ]:
clf = SVC(kernel='rbf', C=1.0 , cache_size=4000, decision_function_shape='ovo')
clf.fit(X_train, y_train)
pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))